In [26]:
import re
import warnings
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import xlsxwriter
from openpyxl import Workbook
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
import statsmodels.api as sm

In [2]:
# 1. Chargement et suppression de la ligne description
df = pd.read_excel('../data/Optimum HLPV 11052025 anonymisé.xlsx')
df_raw = df.iloc[1:].copy()

In [3]:
# 2. Suppression des colonnes sans données
drop_cols = [
    'ALERTE_ORANGE_PREALABLE_A_1e_REHOSPITALISATION_POUR_IC',
    'ALERTE_ROUGE_PREALABLE_A_1e_REHOSPITALISATION_POUR_IC',
    'EHPAD',
    'DOMICILE'
]
df_raw = df_raw.drop(columns=drop_cols)

# 3. Renommage de quelques colonnes
df_raw = df_raw.rename(columns={
    'PRESENCE_ISGLT2_FEVG>50%': 'PRESENCE_ISGLT2_FEVG_sup50',
    '>/=3_FANTASTIQUES_FEVG<50%': 'FANTASTIQUES_3_FEVG_inf50',
    'ANEMIE_HB<10g/l':          'ANEMIE_HB_10_gl',
    'PRESENCE_BB-':            'PRESENCE_BB',
    'PRESENCE_IECi/ARA2':      'PRESENCE_IECi_ARA2'
})

In [4]:
# 4. Définition des listes de variables par type
char_cols    = ['NUMERO_PATIENT']

integer_cols = ['AGE','DMS','SCORE_CHARLSON','NT_PRO_BNP_SORTIE','FERRITINE','DOSAGE_SORTIE_DIURETIC']

double_cols  = ['ADL','IMC','ALBUMINE']

percent_cols = ['HbA1c','CST']

bool_cols    = [
    'SEXE','OBESITE','SYNDROME_DEPRESSIF','TROUBLES_COGNITIFS_SEVERES','ATCD_CANCER','CANCER_ACTIF',
    'ATCD_BPCO','ATCD_HTA','ATCD_DYSLIPIDEMIE','ATCD_DIABETE','ATCD_AVC','ATCD_AOMI','ATCD_FA_FLUTTER',
    'ATCD_AMYLOSE','PM_DAI','RAC_AU_MOINS_MODEREE','IM_AU_MOINS_MODEREE','IT_AU_MOINS_MODEREE',
    'CARENCE_MARTIALE_VRAIE','CARENCE_MARTIALE_FONCTIONNELLE','ANEMIE_HB_10_gl','CARENCE_VITAMINE_B9',
    'CARENCE_VITAMINE_B12','CARENCE_VITAMINE_D','PRESENCE_IECi_ARA2','PRESENCE_ACEi','PRESENCE_BB',
    'PRESENCE_MRA','PRESENCE_SORTIE_ISGLT2','PRESENCE_SORTIE_DIURETIC','PRESENCE_SORTIE_ANTICOAGULANT',
    'PRESENCE_SORTIE_AAP','PRESENCE_ISGLT2_FEVG_sup50','FANTASTIQUES_3_FEVG_inf50'
]
ordinal_mappings = {
    'DENUTRITION': {'0':'Risque dénutrition','1':'Dénutrition modérée','2':'Dénutrition sévère','3':'Inconnu'},
    'ATCD_CARDIOPATHIE_ISCHEMIQUE': {'0':'NON','1':'Traitement médical','2':'Angioplastie stent','3':'Pontage'},
    'FEVG_CLASSE': {'0':'Altérée (</=40%)','1':'Moyennement altérée (41-49%)','2':'Préservée (>/=50%)'},
    'STADE_IRC_DFG_SORTIE': {'1':'DFG>90 ml/min','2':'DFG=60-89 ml/min','3A':'45-59 ml/min','3B':'30-44 ml/min','4':'15-29 ml/min','5':'<14 ml/min'},
    'TYPE_DIURETIC': {'0':'FUROSEMIDE','1':'BUMETAMIDE'},
    'TYPE_SUIVI_SATELIA': {'0':'Patient','1':'Satelia'}
}
rehosp_cols = [
    'REHOSPITALISATION_IC_1MOIS','REHOSPITALISATION_IC_3MOIS',
    'REHOSPITALISATION_IC_6MOIS','REHOSPITALISATION_IC_12MOIS',
    'REHOSPITALISATION_IC_>12MOIS'
]

In [5]:
# 5. Application des transformations
df_clean = df_raw.copy()

# → Caractères
for col in char_cols:
    df_clean[col] = df_clean[col].astype(str)

# → Doubles (correction : on force en str avant le .str.strip())
for col in double_cols:
    df_clean[col] = pd.to_numeric(
        df_clean[col].astype(str).str.strip().replace('NC', pd.NA),
        errors='coerce'
    )

# → Entiers (extraction de chiffres puis Int64 pour garder les NA)
for col in integer_cols:
    df_clean[col] = df_clean[col].astype(str).str.extract(r'(\d+)', expand=False)
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('Int64')

# → Pourcentages
for col in percent_cols:
    df_clean[col] = df_clean[col].astype(str).str.replace('%','').str.strip().replace('NC', pd.NA)
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# → Booléens en catégories “No”/“Yes”
bool_map = {'1':True,'0':False,1:True,0:False,True:True,False:False}
for col in bool_cols:
    df_clean[col] = df_clean[col].map(bool_map)
    df_clean[col] = df_clean[col].map({False:'No',True:'Yes'}).astype('category')

# → Réhospitalisations
for col in rehosp_cols:
    # 1) on force en numérique (les NC, '', etc. deviennent NaN)
    num = pd.to_numeric(df_clean[col], errors='coerce')
    # 2) on mappe 1→Yes, 0→No
    df_clean[col] = num.map({1:'Yes', 0:'No'}).astype('category')

# → Variables ordinales
for col, mapping in ordinal_mappings.items():
    cat_type = CategoricalDtype(categories=list(mapping.values()), ordered=True)
    df_clean[col] = df_clean[col].astype(str).map(mapping).astype(cat_type)


In [6]:
# 6. Vérification rapide
df_clean.head()

,NUMERO_PATIENT,AGE,SEXE,DMS,ADL,IMC,OBESITE,DENUTRITION,SCORE_CHARLSON,SYNDROME_DEPRESSIF,...,PRESENCE_SORTIE_ANTICOAGULANT,PRESENCE_SORTIE_AAP,TYPE_SUIVI_SATELIA,OUTCOME_REHOSPITALISATION_12MOIS,REHOSPITALISATION_IC_1MOIS,REHOSPITALISATION_IC_3MOIS,REHOSPITALISATION_IC_6MOIS,REHOSPITALISATION_IC_12MOIS,REHOSPITALISATION_IC_>12MOIS,NBRE_REHOSPITALISATION_IC_12MOIS
1,1.0,94,Yes,6,5.5,25.5,No,Dénutrition sévère,9,No,...,No,Yes,Patient,0,No,No,No,No,Yes,0
2,2.0,77,Yes,7,6.0,22.4,No,Dénutrition sévère,7,No,...,Yes,No,Patient,0,No,No,No,No,No,0
3,3.0,77,No,8,5.5,32.3,Yes,Dénutrition sévère,7,No,...,Yes,No,Satelia,1,No,No,No,Yes,No,3
4,4.0,90,No,8,5.0,28.6,No,Dénutrition sévère,6,No,...,No,No,Satelia,0,No,No,No,No,No,0
5,5.0,91,No,7,5.5,21.0,No,Dénutrition sévère,6,No,...,Yes,No,Patient,1,No,Yes,Yes,Yes,No,2


In [7]:
print(df_clean.dtypes)

NUMERO_PATIENT                        object
AGE                                    Int64
SEXE                                category
DMS                                    Int64
ADL                                  float64
IMC                                  float64
OBESITE                             category
DENUTRITION                         category
SCORE_CHARLSON                         Int64
SYNDROME_DEPRESSIF                  category
TROUBLES_COGNITIFS_SEVERES          category
ATCD_CANCER                         category
CANCER_ACTIF                        category
ATCD_BPCO                           category
ATCD_HTA                            category
ATCD_DYSLIPIDEMIE                   category
ATCD_DIABETE                        category
ATCD_AVC                            category
ATCD_AOMI                           category
ATCD_CARDIOPATHIE_ISCHEMIQUE        category
ATCD_FA_FLUTTER                     category
ATCD_AMYLOSE                        category
PM_DAI    

In [8]:
# Fonction pour calculer la proportion de NA et ne conserver que les colonnes à NA > 0
def na_props(df, cols):
    # isna().mean() donne la proportion de NA par colonne
    props = df[cols].isna().mean()
    return props[props > 0]

# Appliquer à chaque groupe de colonnes
integer_na   = na_props(df_clean, integer_cols)
double_na    = na_props(df_clean, double_cols)
percent_na   = na_props(df_clean, percent_cols)
bool_na      = na_props(df_clean, bool_cols)
rehosp_na    = na_props(df_clean, rehosp_cols)
ordinal_na   = na_props(df_clean, list(ordinal_mappings.keys()))

# Afficher le résultat
missing_summary = {
    'integer'          : integer_na,
    'double'           : double_na,
    'percent'          : percent_na,
    'boolean'          : bool_na,
    'rehospitalisation': rehosp_na,
    'ordinal'          : ordinal_na
}

for group, series in missing_summary.items():
    print(f"\n– {group} cols with NA > 0:")
    print(series)



– integer cols with NA > 0:
NT_PRO_BNP_SORTIE         0.022222
FERRITINE                 0.066667
DOSAGE_SORTIE_DIURETIC    0.059259
dtype: float64

– double cols with NA > 0:
Series([], dtype: float64)

– percent cols with NA > 0:
HbA1c    0.200000
CST      0.081481
dtype: float64

– boolean cols with NA > 0:
RAC_AU_MOINS_MODEREE              0.007407
IM_AU_MOINS_MODEREE               0.007407
IT_AU_MOINS_MODEREE               0.007407
CARENCE_MARTIALE_VRAIE            0.066667
CARENCE_MARTIALE_FONCTIONNELLE    0.066667
CARENCE_VITAMINE_B9               0.037037
CARENCE_VITAMINE_B12              0.037037
CARENCE_VITAMINE_D                0.007407
dtype: float64

– rehospitalisation cols with NA > 0:
Series([], dtype: float64)

– ordinal cols with NA > 0:
FEVG_CLASSE      0.014815
TYPE_DIURETIC    0.059259
dtype: float64


In [9]:
# Variables ordinales (noms uniquement)
ordinal_vars = [
    'DENUTRITION', 'ATCD_CARDIOPATHIE_ISCHEMIQUE', 'FEVG_CLASSE',
    'STADE_IRC_DFG_SORTIE', 'TYPE_DIURETIC', 'TYPE_SUIVI_SATELIA'
]

# Variables "spéciales"
special_vars = [
    'PRESENCE_ISGLT2_FEVG_sup50', 'FANTASTIQUES_3_FEVG_inf50'
]

# On retire special_vars de la liste des booléens
bool_wo_special = [col for col in bool_cols if col not in special_vars]

# Variables continues
cont_vars = double_cols + integer_cols + percent_cols

# Variables catégorielles (booléennes non spéciales + ordinales + spéciales)
cat_vars_initial = bool_wo_special + ordinal_vars + special_vars

# 1. Vos vraies binaires (sans les ordinales)
cat_vars = [col for col in cat_vars_initial if col not in ordinal_vars]

all_vars = cont_vars + cat_vars + ordinal_vars

horizons = rehosp_cols[:4]

# Affichage des listes
print("cont_vars:", cont_vars)
print("\ncat_vars_all:", cat_vars_initial )
print("\nhorizons:", horizons)

cont_vars: ['ADL', 'IMC', 'ALBUMINE', 'AGE', 'DMS', 'SCORE_CHARLSON', 'NT_PRO_BNP_SORTIE', 'FERRITINE', 'DOSAGE_SORTIE_DIURETIC', 'HbA1c', 'CST']

cat_vars_all: ['SEXE', 'OBESITE', 'SYNDROME_DEPRESSIF', 'TROUBLES_COGNITIFS_SEVERES', 'ATCD_CANCER', 'CANCER_ACTIF', 'ATCD_BPCO', 'ATCD_HTA', 'ATCD_DYSLIPIDEMIE', 'ATCD_DIABETE', 'ATCD_AVC', 'ATCD_AOMI', 'ATCD_FA_FLUTTER', 'ATCD_AMYLOSE', 'PM_DAI', 'RAC_AU_MOINS_MODEREE', 'IM_AU_MOINS_MODEREE', 'IT_AU_MOINS_MODEREE', 'CARENCE_MARTIALE_VRAIE', 'CARENCE_MARTIALE_FONCTIONNELLE', 'ANEMIE_HB_10_gl', 'CARENCE_VITAMINE_B9', 'CARENCE_VITAMINE_B12', 'CARENCE_VITAMINE_D', 'PRESENCE_IECi_ARA2', 'PRESENCE_ACEi', 'PRESENCE_BB', 'PRESENCE_MRA', 'PRESENCE_SORTIE_ISGLT2', 'PRESENCE_SORTIE_DIURETIC', 'PRESENCE_SORTIE_ANTICOAGULANT', 'PRESENCE_SORTIE_AAP', 'DENUTRITION', 'ATCD_CARDIOPATHIE_ISCHEMIQUE', 'FEVG_CLASSE', 'STADE_IRC_DFG_SORTIE', 'TYPE_DIURETIC', 'TYPE_SUIVI_SATELIA', 'PRESENCE_ISGLT2_FEVG_sup50', 'FANTASTIQUES_3_FEVG_inf50']

horizons: ['REHOSPITA

In [10]:
print(cat_vars)

['SEXE', 'OBESITE', 'SYNDROME_DEPRESSIF', 'TROUBLES_COGNITIFS_SEVERES', 'ATCD_CANCER', 'CANCER_ACTIF', 'ATCD_BPCO', 'ATCD_HTA', 'ATCD_DYSLIPIDEMIE', 'ATCD_DIABETE', 'ATCD_AVC', 'ATCD_AOMI', 'ATCD_FA_FLUTTER', 'ATCD_AMYLOSE', 'PM_DAI', 'RAC_AU_MOINS_MODEREE', 'IM_AU_MOINS_MODEREE', 'IT_AU_MOINS_MODEREE', 'CARENCE_MARTIALE_VRAIE', 'CARENCE_MARTIALE_FONCTIONNELLE', 'ANEMIE_HB_10_gl', 'CARENCE_VITAMINE_B9', 'CARENCE_VITAMINE_B12', 'CARENCE_VITAMINE_D', 'PRESENCE_IECi_ARA2', 'PRESENCE_ACEi', 'PRESENCE_BB', 'PRESENCE_MRA', 'PRESENCE_SORTIE_ISGLT2', 'PRESENCE_SORTIE_DIURETIC', 'PRESENCE_SORTIE_ANTICOAGULANT', 'PRESENCE_SORTIE_AAP', 'PRESENCE_ISGLT2_FEVG_sup50', 'FANTASTIQUES_3_FEVG_inf50']


In [11]:
print(ordinal_vars)

['DENUTRITION', 'ATCD_CARDIOPATHIE_ISCHEMIQUE', 'FEVG_CLASSE', 'STADE_IRC_DFG_SORTIE', 'TYPE_DIURETIC', 'TYPE_SUIVI_SATELIA']


In [12]:
def univariate_summary(
    df: pd.DataFrame,
    cont_vars: list,
    cat_vars: list,
    ordinal_vars: list,
    horizons: list
) -> dict:
    """
    Univariate summary table for each rehospitalization horizon.

    Parameters
    ----------
    df : DataFrame
        Cleaned dataset with 'Yes'/'No' in horizon columns.
    cont_vars : list of str
        Names of continuous variables.
    cat_vars : list of str
        Names of binary categorical variables.
    ordinal_vars : list of str
        Names of ordinal variables (pd.Categorical with ordered categories).
    horizons : list of str
        Names of rehospitalization horizon columns.

    Returns
    -------
    dict
        Mapping each horizon to a DataFrame with columns:
        ['Variable','Level','Missing','Overall','Yes','No','CI_95','p_value']
    """
    summaries = {}
    N = len(df)

    for h in horizons:
        mask_yes = df[h] == 'Yes'
        mask_no  = df[h] == 'No'
        n_yes    = mask_yes.sum()
        n_no     = mask_no.sum()
        n_tot    = n_yes + n_no

        # Horizon-level missing
        n_miss_h   = df[h].isna().sum()
        pct_miss_h = n_miss_h / N * 100

        rows = []
        # Total row
        rows.append({
            'Variable': 'Total',
            'Level':    '',
            'Missing':  f"{n_miss_h} ({pct_miss_h:.1f}%)",
            'Overall':  f"{n_tot}",
            'Yes':      f"{n_yes} ({(n_yes/n_tot*100) if n_tot>0 else 0:.1f}%)",
            'No':       f"{n_no} ({(n_no/n_tot*100) if n_tot>0 else 0:.1f}%)",
            'CI_95':    '',
            'p_value':  ''
        })

        # Continuous variables
        for var in cont_vars:
            col = df[var]
            n_na = col.isna().sum()
            pct_na = col.isna().mean() * 100
            missing = f"{n_na} ({pct_na:.1f}%)"

            x_all = col.dropna()
            x_yes = col[mask_yes].dropna()
            x_no  = col[mask_no].dropna()

            m_all, s_all = x_all.mean(), x_all.std()
            m_yes, s_yes = x_yes.mean(), x_yes.std()
            m_no,  s_no  = x_no.mean(),  x_no.std()

            overall = f"{m_all:.2f} ± {s_all:.2f}"
            yes_str  = f"{m_yes:.2f} ± {s_yes:.2f}"
            no_str   = f"{m_no:.2f} ± {s_no:.2f}"

            # p-value from t-test
            p_val = np.nan
            try:
                p_val = stats.ttest_ind(x_yes, x_no, equal_var=False, nan_policy='omit').pvalue
            except:
                pass
            p_str = "<0.001" if (not np.isnan(p_val) and p_val < 0.001) else (f"{p_val:.3f}" if not np.isnan(p_val) else "")

            # CI 95% around the Yes-group mean
            ci_str = ""
            if len(x_yes) > 1:
                se_yes = s_yes / np.sqrt(len(x_yes))
                tcrit  = stats.t.ppf(0.975, df=len(x_yes)-1)
                lo = m_yes - tcrit * se_yes
                hi = m_yes + tcrit * se_yes
                ci_str = f"{lo:.2f}–{hi:.2f}"

            rows.append({
                'Variable': var,
                'Level':    '',
                'Missing':  missing,
                'Overall':  overall,
                'Yes':      yes_str,
                'No':       no_str,
                'CI_95':    ci_str,
                'p_value':  p_str
            })

        # Ordinal variables: one row per level, with Wilson CI
        for var in ordinal_vars:
            col = df[var]
            n_na = col.isna().sum()
            pct_na = col.isna().mean() * 100
            missing = f"{n_na} ({pct_na:.1f}%)"

            # Global p-value via chi² for the entire variable
            tbl = pd.crosstab(col, df[h])
            try:
                p_global = stats.chi2_contingency(tbl)[1]
            except:
                p_global = np.nan
            p_str_global = "<0.001" if (not np.isnan(p_global) and p_global < 0.001) else (f"{p_global:.3f}" if not np.isnan(p_global) else "")

            first = True
            n_obs = col.notna().sum()

            for lvl in df[var].cat.categories:
                count_lvl = (col == lvl).sum()
                pct_lvl   = count_lvl / n_obs * 100 if n_obs > 0 else 0

                # Wilson CI for this level
                lo, hi = proportion_confint(count_lvl, n_obs, alpha=0.05, method='wilson')
                ci_lvl = f"{lo*100:.1f}%–{hi*100:.1f}%"

                count_yes = ((col == lvl) & mask_yes).sum()
                pct_yes_lvl = count_yes / n_yes * 100 if n_yes > 0 else 0

                count_no  = ((col == lvl) & mask_no).sum()
                pct_no_lvl = count_no / n_no * 100 if n_no > 0 else 0

                rows.append({
                    'Variable':       var,
                    'Level':          lvl,
                    'Missing':        missing if first else '',
                    'Overall':        f"{count_lvl} ({pct_lvl:.1f}%)",
                    'Yes':            f"{count_yes} ({pct_yes_lvl:.1f}%)",
                    'No':             f"{count_no} ({pct_no_lvl:.1f}%)",
                    'CI_95':          ci_lvl,
                    'p_value':        p_str_global if first else ''
                })
                first = False

        # Binary categorical variables
        for var in cat_vars:
            col = df[var]
            n_na = col.isna().sum()
            pct_na = col.isna().mean() * 100
            missing = f"{n_na} ({pct_na:.1f}%)"

            x_all = (col == 'Yes').sum()
            n_all = col.notna().sum()
            pct_all = x_all / n_all * 100 if n_all > 0 else 0

            x_yes = ((col == 'Yes') & mask_yes).sum()
            pct_yes_l = x_yes / n_yes * 100 if n_yes > 0 else 0

            x_no  = ((col == 'Yes') & mask_no).sum()
            pct_no_l = x_no / n_no * 100 if n_no > 0 else 0

            overall = f"{x_all} ({pct_all:.1f}%)"
            yes_str = f"{x_yes} ({pct_yes_l:.1f}%)"
            no_str  = f"{x_no} ({pct_no_l:.1f}%)"

            # Wilson CI for overall proportion
            ci_cat = ""
            if n_all > 0:
                lo, hi = proportion_confint(x_all, n_all, alpha=0.05, method='wilson')
                ci_cat = f"{lo*100:.1f}%–{hi*100:.1f}%"

            # p-value from Fisher or χ²
            tbl = pd.crosstab(col, df[h])
            try:
                if tbl.shape == (2,2):
                    ptest = stats.fisher_exact(tbl.values)[1]
                else:
                    ptest = stats.chi2_contingency(tbl)[1]
            except:
                ptest = np.nan
            p_str = "<0.001" if (not np.isnan(ptest) and ptest < 0.001) else (f"{ptest:.3f}" if not np.isnan(ptest) else "")

            rows.append({
                'Variable': var,
                'Level':    '',
                'Missing':  missing,
                'Overall':  overall,
                'Yes':      yes_str,
                'No':       no_str,
                'CI_95':    ci_cat,
                'p_value':  p_str
            })

        summaries[h] = pd.DataFrame(rows)

    return summaries

In [13]:
results_uni = univariate_summary(
    df_clean,
    cont_vars=cont_vars,
    cat_vars=cat_vars,         
    ordinal_vars=ordinal_vars,   
    horizons=horizons
)
results_uni

{'REHOSPITALISATION_IC_1MOIS':                          Variable Level   Missing       Overall           Yes  \
 0                           Total        0 (0.0%)           135      5 (3.7%)   
 1                             ADL        0 (0.0%)   5.27 ± 0.98   5.80 ± 0.27   
 2                             IMC        0 (0.0%)  26.90 ± 5.42  26.20 ± 3.98   
 3                        ALBUMINE        0 (0.0%)  31.40 ± 5.24  31.94 ± 2.43   
 4                             AGE        0 (0.0%)  85.84 ± 6.82  86.00 ± 4.00   
 ..                            ...   ...       ...           ...           ...   
 62       PRESENCE_SORTIE_DIURETIC        0 (0.0%)   127 (94.1%)     4 (80.0%)   
 63  PRESENCE_SORTIE_ANTICOAGULANT        0 (0.0%)   103 (76.3%)     3 (60.0%)   
 64            PRESENCE_SORTIE_AAP        0 (0.0%)    29 (21.5%)     2 (40.0%)   
 65     PRESENCE_ISGLT2_FEVG_sup50        0 (0.0%)    76 (56.3%)     3 (60.0%)   
 66      FANTASTIQUES_3_FEVG_inf50        0 (0.0%)    33 (24.4%)    

In [14]:
results_uni["REHOSPITALISATION_IC_12MOIS"].head(60)

,Variable,Level,Missing,Overall,Yes,No,CI_95,p_value
0,Total,,0 (0.0%),135,29 (21.5%),106 (78.5%),,
1,ADL,,0 (0.0%),5.27 ± 0.98,5.05 ± 1.22,5.33 ± 0.90,4.59–5.52,0.259
2,IMC,,0 (0.0%),26.90 ± 5.42,27.45 ± 4.77,26.75 ± 5.59,25.64–29.27,0.501
3,ALBUMINE,,0 (0.0%),31.40 ± 5.24,33.01 ± 3.64,30.96 ± 5.53,31.62–34.39,0.021
4,AGE,,0 (0.0%),85.84 ± 6.82,86.41 ± 6.40,85.69 ± 6.96,83.98–88.85,0.598
5,DMS,,0 (0.0%),7.52 ± 3.19,7.14 ± 2.81,7.62 ± 3.29,6.07–8.21,0.432
6,SCORE_CHARLSON,,0 (0.0%),7.97 ± 2.30,8.72 ± 2.07,7.76 ± 2.32,7.94–9.51,0.036
7,NT_PRO_BNP_SORTIE,,3 (2.2%),3200.90 ± 4726.57,3284.83 ± 3287.50,3177.27 ± 5071.77,2034.33–4535.32,0.892
8,FERRITINE,,9 (6.7%),350.06 ± 385.96,234.34 ± 380.35,384.66 ± 382.78,89.67–379.02,0.069
9,DOSAGE_SORTIE_DIURETIC,,8 (5.9%),113.66 ± 163.10,170.18 ± 222.65,97.68 ± 139.19,83.85–256.51,0.111


In [15]:
for h in horizons:
    print(f"Horizon {h!r} → modalities:", df_clean[h].value_counts(dropna=False).to_dict())


Horizon 'REHOSPITALISATION_IC_1MOIS' → modalities: {'No': 130, 'Yes': 5}
Horizon 'REHOSPITALISATION_IC_3MOIS' → modalities: {'No': 124, 'Yes': 11}
Horizon 'REHOSPITALISATION_IC_6MOIS' → modalities: {'No': 116, 'Yes': 19}
Horizon 'REHOSPITALISATION_IC_12MOIS' → modalities: {'No': 106, 'Yes': 29}


In [16]:
confounders  = ["AGE", "SEXE", "ATCD_HTA", "SCORE_CHARLSON", 
              "IM_AU_MOINS_MODEREE", "CARENCE_MARTIALE_VRAIE"]

In [17]:
def multivariate_pvalues(df, vars_multivariees, horizon):
    """
    Ajuste un modèle logit multivarié avec toutes les vars_multivariees
    et renvoie DataFrame(variable, OR, CI_lower, CI_upper, p_value_multi).
    """
    # 1) Prépare données
    dfm = df.dropna(subset=vars_multivariees + [horizon]).copy()
    y   = dfm[horizon].map({'Yes':1,'No':0})
    # 2) Design matrix
    X   = pd.get_dummies(dfm[vars_multivariees], drop_first=True)
    X   = sm.add_constant(X).astype(float)
    # 3) Fit logit
    result = sm.Logit(y, X).fit(disp=False)
    # 4) Extrait
    params = result.params
    conf   = result.conf_int()
    pvals  = result.pvalues
    out = pd.DataFrame({
        'variable'     : params.index,
        'OR'           : np.exp(params.values),
        'CI_lower'     : np.exp(conf[0]),
        'CI_upper'     : np.exp(conf[1]),
        'p_value_multi': [
            "<0.001" if p<0.001 else f"{p:.3f}"
            for p in pvals
        ]
    }).reset_index(drop=True)
    return out

In [18]:
multiv_df = multivariate_pvalues(df_clean, confounders , "REHOSPITALISATION_IC_12MOIS")
display(multiv_df)

,variable,OR,CI_lower,CI_upper,p_value_multi
0,const,0.158213,0.000098,256.006656,0.625
1,AGE,0.961963,0.885496,1.045034,0.359
2,SCORE_CHARLSON,1.377655,1.076337,1.763326,0.011
3,SEXE_Yes,0.535195,0.174175,1.644519,0.275
4,ATCD_HTA_Yes,2.211390,0.525002,9.314721,0.279
5,IM_AU_MOINS_MODEREE_Yes,34.417038,2.957254,400.551445,0.005
6,CARENCE_MARTIALE_VRAIE_Yes,7.017094,2.356119,20.898613,<0.001


In [19]:
def merge_univ_with_adjusted_multiv(
    uni_results: dict,
    df: pd.DataFrame,
    confounders: list,
    all_vars: list,
    horizons: list
) -> dict:
    """
    Pour chaque horizon, ajoute une colonne p_value_multi à la table univariée,
    en ajustant chaque variable d'intérêt sur la liste des confounders (sans elle)
    via un mini-modèle GLM Binomial (IRLS).
    """
    merged = {}
    for h in horizons:
        # on ne garde que Yes/No
        dfh = df[df[h].isin(['Yes','No'])].copy()
        dfh['y'] = dfh[h].map({'Yes':1,'No':0})
        
        p_map = {}
        for var in all_vars:
            # on retire var de la liste des confounders
            conf_for_var = [c for c in confounders if c != var]
            # on construit un DataFrame minimal et on dropna uniquement sur var et y
            dmod = dfh.dropna(subset=[var,'y'])
            if dmod.empty:
                p_map[var] = ""
                continue
            
            # design matrix : continue et dummies seulement sur les confounders + var
            X = dmod[[var] + conf_for_var].copy()
            X = pd.get_dummies(X, drop_first=True)
            X = sm.add_constant(X).astype(float)
            y = dmod['y']
            
            try:
                # GLM Binomial (IRLS), plus robuste que Logit.fit()
                model = sm.GLM(y, X, family=sm.families.Binomial())
                res   = model.fit(disp=False, maxiter=100)
            except Exception:
                p_map[var] = ""
                continue
            
            # on cherche le nom du coefficient qui correspond à var
            # (soit 'var' pour continu, soit 'var_level' pour dummies)
            candidates = [c for c in res.pvalues.index if c == var or c.startswith(var + '_')]
            if not candidates:
                p_map[var] = ""
            else:
                p = res.pvalues[candidates[0]]
                p_map[var] = "<0.001" if p < 0.001 else f"{p:.3f}"
        
        # on fusionne la colonne p_value_multi dans la table univariée
        uni_df = uni_results[h].copy()
        uni_df['p_value_multi'] = uni_df['Variable'].map(p_map).fillna("")
        merged[h] = uni_df

    return merged

In [20]:
results_adj = merge_univ_with_adjusted_multiv(results_uni, 
                                              df_clean, 
                                              confounders, 
                                              all_vars,
                                              horizons
                                             )

results_adj['REHOSPITALISATION_IC_12MOIS'].head(60)

,Variable,Level,Missing,Overall,Yes,No,CI_95,p_value,p_value_multi
0,Total,,0 (0.0%),135,29 (21.5%),106 (78.5%),,,
1,ADL,,0 (0.0%),5.27 ± 0.98,5.05 ± 1.22,5.33 ± 0.90,4.59–5.52,0.259,0.291
2,IMC,,0 (0.0%),26.90 ± 5.42,27.45 ± 4.77,26.75 ± 5.59,25.64–29.27,0.501,0.961
3,ALBUMINE,,0 (0.0%),31.40 ± 5.24,33.01 ± 3.64,30.96 ± 5.53,31.62–34.39,0.021,0.468
4,AGE,,0 (0.0%),85.84 ± 6.82,86.41 ± 6.40,85.69 ± 6.96,83.98–88.85,0.598,0.684
5,DMS,,0 (0.0%),7.52 ± 3.19,7.14 ± 2.81,7.62 ± 3.29,6.07–8.21,0.432,0.581
6,SCORE_CHARLSON,,0 (0.0%),7.97 ± 2.30,8.72 ± 2.07,7.76 ± 2.32,7.94–9.51,0.036,0.007
7,NT_PRO_BNP_SORTIE,,3 (2.2%),3200.90 ± 4726.57,3284.83 ± 3287.50,3177.27 ± 5071.77,2034.33–4535.32,0.892,0.635
8,FERRITINE,,9 (6.7%),350.06 ± 385.96,234.34 ± 380.35,384.66 ± 382.78,89.67–379.02,0.069,0.996
9,DOSAGE_SORTIE_DIURETIC,,8 (5.9%),113.66 ± 163.10,170.18 ± 222.65,97.68 ± 139.19,83.85–256.51,0.111,0.007


In [21]:
def split_special_tables_with_total_and_suffix(
    results_adj: dict,
    df: pd.DataFrame,
    special_vars: list,
    cont_vars: list
) -> dict:
    """
    Pour chaque horizon, retourne :
      - 'global' : toutes les variables sauf special_vars, AVEC la ligne Total en tête
      - une table par variable spéciale, AVEC la ligne Total en tête et les niveaux No/Yes
    Et suffixe " (Mean (SD))" après le nom des variables continues (Level == '').
    """
    split = {}
    for h, uni_df in results_adj.items():
        # Extraire la ligne Total
        total_row = uni_df[uni_df['Variable'] == 'Total'].iloc[0]

        # --- 1) Table globale avec Total ---
        df_global = uni_df[~uni_df['Variable'].isin(special_vars)].copy()
        df_global = pd.concat([
            pd.DataFrame([total_row]),
            df_global[df_global['Variable'] != 'Total']
        ], ignore_index=True)

        # 2) Suffixe " (Mean (SD))" pour les variables continues
        mask_cont = df_global['Level'].eq('') & df_global['Variable'].isin(cont_vars)
        df_global.loc[mask_cont, 'Variable'] = (
            df_global.loc[mask_cont, 'Variable'] + " (Mean (SD))"
        )

        # --- 3) Tables spéciales ---
        special_tables = {}
        dfh = df[df[h].isin(['Yes','No'])].copy()
        for var in special_vars:
            rows = [total_row.to_dict()]
            dmod = dfh.dropna(subset=[var])
            n_obs = dmod[var].notna().sum()
            mask_yes = dmod[h] == 'Yes'
            mask_no  = dmod[h] == 'No'

            missing_var = uni_df.loc[uni_df['Variable']==var, 'Missing'].iat[0]
            p_val      = uni_df.loc[uni_df['Variable']==var, 'p_value'].iat[0]
            p_adj      = uni_df.loc[uni_df['Variable']==var, 'p_value_multi'].iat[0]

            for level in ['No','Yes']:
                count_lvl = (dmod[var] == level).sum()
                pct_lvl   = count_lvl / n_obs * 100 if n_obs>0 else 0
                overall   = f"{count_lvl} ({pct_lvl:.1f}%)"

                x_yes = ((dmod[var]==level) & mask_yes).sum()
                pct_yes = x_yes / mask_yes.sum() * 100 if mask_yes.sum()>0 else 0
                yes_str = f"{x_yes} ({pct_yes:.1f}%)"

                x_no = ((dmod[var]==level) & mask_no).sum()
                pct_no = x_no / mask_no.sum() * 100 if mask_no.sum()>0 else 0
                no_str = f"{x_no} ({pct_no:.1f}%)"

                lo, hi = proportion_confint(count_lvl, n_obs, alpha=0.05, method='wilson')
                ci_str = f"{lo*100:.1f}%–{hi*100:.1f}%"

                rows.append({
                    'Variable'    : var,
                    'Level'       : level,
                    'Missing'     : missing_var,
                    'Overall'     : overall,
                    'Yes'         : yes_str,
                    'No'          : no_str,
                    'CI_95'       : ci_str,
                    'p_value'     : p_val if level=='Yes' else "",
                    'p_value_multi' : p_adj if level=='Yes' else ""
                })

            df_sp = pd.DataFrame(rows)
            # Suffixe pour le cas improbable d'une var continue en special_vars
            mask_cont_sp = df_sp['Level'].eq('') & df_sp['Variable'].isin(cont_vars)
            df_sp.loc[mask_cont_sp, 'Variable'] = (
                df_sp.loc[mask_cont_sp, 'Variable'] + " Mean (SD)"
            )

            special_tables[var] = df_sp.reset_index(drop=True)

        split[h] = {'global': df_global, **special_tables}

    return split

split_tables = split_special_tables_with_total_and_suffix(results_adj, df_clean, special_vars, cont_vars)

# Pour accéder aux 3 tableaux de 12 mois :
global_12m   = split_tables['REHOSPITALISATION_IC_12MOIS']['global']
isglt2_12m   = split_tables['REHOSPITALISATION_IC_12MOIS']['PRESENCE_ISGLT2_FEVG_sup50']
fant_12m     = split_tables['REHOSPITALISATION_IC_12MOIS']['FANTASTIQUES_3_FEVG_inf50']

global_12m, isglt2_12m, fant_12m


(                         Variable Level   Missing       Overall           Yes  \
 0                           Total        0 (0.0%)           135    29 (21.5%)   
 1                 ADL (Mean (SD))        0 (0.0%)   5.27 ± 0.98   5.05 ± 1.22   
 2                 IMC (Mean (SD))        0 (0.0%)  26.90 ± 5.42  27.45 ± 4.77   
 3            ALBUMINE (Mean (SD))        0 (0.0%)  31.40 ± 5.24  33.01 ± 3.64   
 4                 AGE (Mean (SD))        0 (0.0%)  85.84 ± 6.82  86.41 ± 6.40   
 ..                            ...   ...       ...           ...           ...   
 60                   PRESENCE_MRA        0 (0.0%)    21 (15.6%)     5 (17.2%)   
 61         PRESENCE_SORTIE_ISGLT2        0 (0.0%)   118 (87.4%)    25 (86.2%)   
 62       PRESENCE_SORTIE_DIURETIC        0 (0.0%)   127 (94.1%)    28 (96.6%)   
 63  PRESENCE_SORTIE_ANTICOAGULANT        0 (0.0%)   103 (76.3%)    22 (75.9%)   
 64            PRESENCE_SORTIE_AAP        0 (0.0%)    29 (21.5%)     6 (20.7%)   
 
              

In [28]:
def export_to_excel_with_formatting(split_tables: dict, filename: str):
    """
    Exporte les tables globales et spéciales pour chaque horizon dans un fichier Excel,
    avec un onglet par horizon (1mois, 3mois, 6mois, 12mois),
    et met en gras les cellules de p_value et p_value_adj <0.02 ou "<0.001".
    """
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        workbook  = writer.book
        # Format for bold
        bold_fmt = workbook.add_format({'bold': True})
        
        for horizon, tables in split_tables.items():
            sheet_name = horizon.replace("REHOSPITALISATION_IC_", "").replace("MOIS", "mois")
            worksheet = workbook.add_worksheet(sheet_name)
            writer.sheets[sheet_name] = worksheet
            
            startrow = 0
            for name, df in tables.items():
                # Title
                title = "Global" if name == "global" else f"Spéciale: {name}"
                worksheet.write(startrow, 0, title, workbook.add_format({'bold': True}))
                # Write DataFrame
                df.to_excel(writer, sheet_name=sheet_name, startrow=startrow+1, index=False)
                
                # Apply conditional formatting for p_value and p_value_adj columns
                for col_name in ['p_value', 'p_value_adj']:
                    if col_name in df.columns:
                        col_idx = df.columns.get_loc(col_name)
                        first_row = startrow + 1
                        last_row  = startrow + 1 + df.shape[0] - 1
                        # Numeric <0.02
                        worksheet.conditional_format(first_row, col_idx, last_row, col_idx, {
                            'type': 'cell',
                            'criteria': '<',
                            'value': 0.2,
                            'format': bold_fmt
                        })
                        # Text containing "<0.001"
                        worksheet.conditional_format(first_row, col_idx, last_row, col_idx, {
                            'type': 'text',
                            'criteria': 'containing',
                            'value': '<0.001',
                            'format': bold_fmt
                        })
                
                # Move startrow down
                startrow += df.shape[0] + 3
        
    return filename


output_path = export_to_excel_with_formatting(split_tables, '../output/Table_uni_multi_complet_v9.xlsx')
print(f"Fichier enregistré au {output_path}")

Fichier enregistré au ../output/Table_uni_multi_complet_v9.xlsx
